In [1]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

Using TensorFlow backend.


In [2]:
pwd

'/Users/Chanti'

In [3]:
cd '/Users/Chanti/Desktop/Cookbook/Chapter 8'

/Users/Chanti/Desktop/Cookbook/Chapter 8


In [4]:
pwd

'/Users/Chanti/Desktop/Cookbook/Chapter 8'

In [5]:
# load doc into memory
def load_document(name):
    file = open(name, 'r')
    text = file.read()
    file.close()
    return text

In [6]:
# load document
input_filename = 'junglebook.txt'
doc = load_document(input_filename)
print(doc[:2000])

﻿The Project Gutenberg EBook of The Jungle Book, by Rudyard Kipling

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


Title: The Jungle Book

Author: Rudyard Kipling

Release Date: January 16, 2006 [EBook #236]
Last Updated: October 6, 2016

Language: English

Character set encoding: UTF-8

*** START OF THIS PROJECT GUTENBERG EBOOK THE JUNGLE BOOK ***




Produced by An Anonymous Volunteer and David Widger





THE JUNGLE BOOK

By Rudyard Kipling



Contents

     Mowgli’s Brothers
     Hunting-Song of the Seeonee Pack
     Kaa’s Hunting
     Road-Song of the Bandar-Log
     “Tiger! Tiger!”
      Mowgli’s Song
     The White Seal
     Lukannon
     “Rikki-Tikki-Tavi”
      Darzee’s Chant
     Toomai of the Elephants
     Shiv and the Grasshopper
     Her Majesty’s Servants
     Parade So

In [7]:
import string
 
# turn a document into clean tokens
def clean_document(doc):
    doc = doc.replace('--', ' ')
    tokens = doc.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens

In [8]:
# clean document
tokens = clean_document(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['project', 'gutenberg', 'ebook', 'of', 'the', 'jungle', 'book', 'by', 'rudyard', 'kipling', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 'reuse', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'wwwgutenbergorg', 'title', 'the', 'jungle', 'book', 'author', 'rudyard', 'kipling', 'release', 'date', 'january', 'ebook', 'last', 'updated', 'october', 'language', 'english', 'character', 'set', 'encoding', 'start', 'of', 'this', 'project', 'gutenberg', 'ebook', 'the', 'jungle', 'book', 'produced', 'by', 'an', 'anonymous', 'volunteer', 'and', 'david', 'widger', 'the', 'jungle', 'book', 'by', 'rudyard', 'kipling', 'contents', 'brothers', 'huntingsong', 'of', 'the', 'seeonee', 'pack', 'hunting', 'roadsong', 'of', 'the', 'bandarlog', 'song', 'the

In [9]:
# organize into sequences (of length 50) of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 51422


In [10]:
# save tokens to file, one dialog per line
def save_document(lines, name):
    data = '\n'.join(lines)
    file = open(name, 'w')
    file.write(data)
    file.close()

In [11]:
# save sequences to file
output_filename = 'junglebook_sequences.txt'
save_document(sequences, output_filename)

In [12]:
# load document into memory
def load_document(name):
    file = open(name, 'r')
    text = file.read()
    file.close()
    return text
 
# load
input_filename = 'junglebook_sequences.txt'
doc = load_document(input_filename)
lines = doc.split('\n')

In [13]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [14]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1 

In [15]:
# separate into input and output
sequences = array(sequences)
Input, Output = sequences[:,:-1], sequences[:,-1]
Output = to_categorical(Output, num_classes=vocab_size)
sequence_length = Input.shape[1]

In [16]:
# define model
from keras.layers import Dropout
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=sequence_length))
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(200))
model.add(Dropout(0.3))
model.add(Dense(200, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           502800    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 200)           240800    
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_2 (Dense)              (None, 5028)              1010628   
Total params: 2,115,228
Trainable params: 2,115,228
Non-trainable params: 0
_________________________________________________________________


In [17]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(Input, Output, batch_size=250, epochs=75)

Epoch 1/75
51422/51422 [==============================] - 517s 10ms/step - loss: 6.6069 - acc: 0.0682
Epoch 2/75
51422/51422 [==============================] - 501s 10ms/step - loss: 6.2250 - acc: 0.0721
Epoch 3/75
51422/51422 [==============================] - 494s 10ms/step - loss: 6.0805 - acc: 0.0827
Epoch 4/75
51422/51422 [==============================] - 453s 9ms/step - loss: 5.9354 - acc: 0.0911
Epoch 5/75
51422/51422 [==============================] - 451s 9ms/step - loss: 5.8014 - acc: 0.1025
Epoch 6/75
51422/51422 [==============================] - 448s 9ms/step - loss: 5.6800 - acc: 0.1126
Epoch 7/75
51422/51422 [==============================] - 448s 9ms/step - loss: 5.5646 - acc: 0.1198
Epoch 8/75
51422/51422 [==============================] - 447s 9ms/step - loss: 5.4614 - acc: 0.1267
Epoch 9/75
51422/51422 [==============================] - 447s 9ms/step - loss: 5.3677 - acc: 0.1315
Epoch 10/75
51422/51422 [==============================] - 449s 9ms/step - loss: 5.2885 

In [18]:
# save the model to file
model.save('junglebook_trained.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [19]:
# load doc into memory
def load_document(name):
    file = open(name, 'r')
    text = file.read()
    file.close()
    return text
 
# load cleaned text sequences
input_filename = 'junglebook_sequences.txt'
doc = load_document(input_filename)
lines = doc.split('\n')

In [20]:
sequence_length = len(lines[0].split()) - 1

In [21]:
# load the model
from keras.models import load_model
model = load_model('junglebook_trained.h5')

In [22]:
# select a seed text
from random import randint
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

to me not long ago with some rude talk that i was a naked cub and not fit to dig pignuts but i caught tabaqui by the tail and swung him twice against a palmtree to teach him better was foolishness for though tabaqui is a mischiefmaker he would have told



In [23]:
encoded = tokenizer.texts_to_sequences([seed_text])[0]

In [24]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
 
# load doc into memory
def load_document(name):
    file = open(name, 'r')
    text = file.read()
    file.close()
    return text
 
# generate a sequence from a language model
def generate_sequence(model, tokenizer, sequence_length, seed_text, n_words):
	result = list()
	input_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([input_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		prediction = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == prediction:
				out_word = word
				break
		# append to input
		input_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)
 
# load cleaned text sequences
input_filename = 'junglebook_sequences.txt'
doc = load_document(input_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

In [25]:
# load the model
model = load_model('junglebook_trained.h5')
 
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated = generate_sequence(model, tokenizer, sequence_length, seed_text, 50)
print(generated)

baskets of dried grass and put grasshoppers in them or catch two praying mantises and make them fight or string a necklace of red and black jungle nuts or watch a lizard basking on a rock or a snake hunting a frog near the wallows then they sing long long songs

with odd native quavers at the end of the review and the hyaena whom he had seen the truth they feel twitched to the noises round him for a picture of the end of the ravine and snuffing bitten and best of the bulls at the dawn is a native


In [26]:
# load the model
model = load_model('junglebook_trained.h5')
 
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated = generate_sequence(model, tokenizer, sequence_length, seed_text, 50)
print(generated)

little toomai there was a splash and a trample and the rush of running water and kala nag strode through the bed of a river feeling his way at each step above the noise of the water as it swirled round the legs little toomai could hear more splashing and some

trumpeting both upstream and down grass and knocked him up to the jealous moon he could see bruised of dust for the potter was rann caught him up to the plowed din of the melbourne lines where the two wolves would be forced to make themselves rifles and the sparks


In [29]:
# load the model
model = load_model('junglebook_trained.h5')
 
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated = generate_sequence(model, tokenizer, sequence_length, seed_text, 50)
print(generated)

is in their legs and he remembered the good firm beaches of novastoshnah seven thousand miles away the games his companions played the smell of the seaweed the seal roar and the fighting that very minute he turned north swimming steadily and as he went on he met scores of his

mates and bound like the deck of the fighters and harness under his breath and he could not be able to stop a ship and ducked to nag wound up with scores of marble tracery showing all the regiments went twisting his head and shoulders and creepers very seldom shows
